In [ ]:
import tensorflow as tf
import utils
import pandas as pd
from PIL import Image
from pprint import pprint
import matplotlib.pyplot as plt

    
metadata = pd.read_csv("../data/HuBMAP-20-dataset_information.csv")
metadata["id"] = metadata["image_file"].str.rstrip(".tiff")
targets = pd.read_csv("../data/train.csv")
targets = targets.merge(metadata, on="id")
for k in ["image_file", "anatomical_structures_segmention_file", "glomerulus_segmentation_file"]:
    targets[k] = "../data/train/" + targets[k]
data = tf.data.Dataset.from_tensor_slices(dict(targets))
data = data.map(utils.parse_image)
data = data.interleave(
    lambda i: tf.data.Dataset.from_tensor_slices({k: i[k] for k in ("image_tiles", "mask_tiles")}),
    cycle_length=1,
    block_length=1000,
    deterministic=True
)

plt.figure(figsize=(15, 15))
for i, tile in data.take(12).enumerate().as_numpy_iterator():
    for j, k in enumerate(["image_tiles", "mask_tiles"]):
        ax = plt.subplot(6, 4, (i + 1) * (j + 1))
        plt.imshow(tile[k])
        plt.axis("off")